In [28]:
import pandas as pd
import requests

API_KEY = ''

In [30]:
price_data = pd.read_csv('../data/제주전력시장_시장전기가격_실시간가격.csv')
price_da_data = pd.read_csv('../data/제주전력시장_시장전기가격_하루전가격.csv')
status_data = pd.read_csv('../data/제주전력시장_현황데이터.csv')
weather_rdata1 = pd.read_csv('../data/기상실측데이터_1.csv')
weather_rdata2 = pd.read_csv('../data/기상실측데이터_2.csv')
weather_fdata1 = pd.read_csv('../data/기상예측데이터_1.csv')
weather_fdata2 = pd.read_csv('../data/기상예측데이터_2.csv')

In [32]:
price_da_data.rename(columns={'하루전가격(원/kWh)': 'smp_da'}, inplace=True)
price_data.rename(columns={'실시간 임시 가격(원/kWh)': 'smp_rt',
                           '실시간 확정 가격(원/kWh)': 'smp_rc'}, inplace=True)
status_data.rename(columns={'공급능력(kW)': 'supply_power',
                            '현재 수요(kW)': 'present_load',
                            '태양광 발전량kW)': 'power_solar',
                            '풍력 발전량(kW)': 'power_wind',
                            '신재생 발전량 총합(kW)': 'renewable_energy_total',
                            '공급 예비력(kW)': 'supply_capacity', 
                            '운영 예비력(kW)': 'operation_capacity'}, inplace=True)

In [34]:
weather_rdata1['ts'] = pd.to_numeric(weather_rdata1['ts'], errors='coerce')
weather_rdata1 = weather_rdata1.dropna(subset=['ts'])

weather_rdata2['ts'] = pd.to_numeric(weather_rdata2['ts'], errors='coerce')
weather_rdata2 = weather_rdata2.dropna(subset=['ts'])

weather_fdata1['ts'] = pd.to_numeric(weather_fdata1['ts'], errors='coerce')
weather_fdata1 = weather_fdata1.dropna(subset=['ts'])

weather_fdata2['ts'] = pd.to_numeric(weather_fdata2['ts'], errors='coerce')
weather_fdata2 = weather_fdata2.dropna(subset=['ts'])

In [36]:
def combine_actual_weather(date):
    data1 = pd.read_csv('../data/기상실측데이터_1.csv')
    data2 = pd.read_csv('../data/기상실측데이터_2.csv')
    
    actual_weather = requests.get(f'https://research-api.solarkim.com/data/cmpt-2024/actual-weather/{date.strftime('%Y-%m-%d')}', headers={
                            'Authorization': f'Bearer {API_KEY}'
                        }).json()
    
    actual_weather_1 = pd.DataFrame(actual_weather['actual_weather_1'])
    for column in actual_weather_1.columns:
        data1[column] = data1[column].astype(actual_weather_1[column].dtype)
    merged_data1 = pd.concat([data1, actual_weather_1], ignore_index=True)
    merged_data1.to_csv('../data/기상실측데이터_1.csv', index=False)
    
    actual_weather_2 = pd.DataFrame(actual_weather['actual_weather_2'])
    for column in actual_weather_2.columns:
        data2[column] = data2[column].astype(actual_weather_2[column].dtype)
    merged_data2 = pd.concat([data2, actual_weather_2], ignore_index=True)
    merged_data2.to_csv('../data/기상실측데이터_2.csv', index=False)

    print(f'{date.strftime('%Y-%m-%d')} 기상실측1: ', data1.shape, merged_data1.shape)
    if data1.shape == merged_data1.shape: 
        print("No data added!!!")
    print(f'{date.strftime('%Y-%m-%d')} 기상실측2: ', data2.shape, merged_data2.shape)
    if data2.shape == merged_data2.shape: 
        print("No data added!!!")

def combine_weather_forecast(date):
    data1 = pd.read_csv('../data/기상예측데이터_1.csv')
    data2 = pd.read_csv('../data/기상예측데이터_2.csv')
    
    weather_forecast = requests.get(f'https://research-api.solarkim.com/data/cmpt-2024/weather-forecast/{date.strftime('%Y-%m-%d')}', headers={
                            'Authorization': f'Bearer {API_KEY}'
                        }).json()
    
    weather_forecast_1 = pd.DataFrame(weather_forecast['weather_forecast_1'])
    for column in weather_forecast_1.columns:
        data1[column] = data1[column].astype(weather_forecast_1[column].dtype)
    merged_data1 = pd.concat([data1, weather_forecast_1], ignore_index=True)
    merged_data1.to_csv('../data/기상예측데이터_1.csv', index=False)
    
    weather_forecast_2 = pd.DataFrame(weather_forecast['weather_forecast_2'])
    for column in weather_forecast_2.columns:
        data2[column] = data2[column].astype(weather_forecast_2[column].dtype)
    merged_data2 = pd.concat([data2, weather_forecast_2], ignore_index=True)
    merged_data2.to_csv('../data/기상예측데이터_2.csv', index=False)

    print(f'{date.strftime('%Y-%m-%d')} 기상예측1: ', data1.shape, merged_data1.shape)
    if data1.shape == merged_data1.shape: 
        print("No data added!!!")
    print(f'{date.strftime('%Y-%m-%d')} 기상예측2: ', data2.shape, merged_data2.shape)
    if data2.shape == merged_data1.shape: 
        print("No data added!!!")

def combine_smp_da(date):
    data = pd.read_csv('../data/제주전력시장_시장전기가격_하루전가격.csv')
    
    smp_da = requests.get(f'https://research-api.solarkim.com/data/cmpt-2024/smp-da/{date.strftime('%Y-%m-%d')}', headers={
                            'Authorization': f'Bearer {API_KEY}'
                        }).json()
    
    smp_da = pd.DataFrame(smp_da)
    for column in smp_da.columns:
        data[column] = data[column].astype(smp_da[column].dtype)
    merged_data = pd.concat([data, smp_da], ignore_index=True)
    merged_data.to_csv('../data/제주전력시장_시장전기가격_하루전가격.csv', index=False)

    print(f'{date.strftime('%Y-%m-%d')} 하루전: ', data.shape, merged_data.shape)
    if data.shape == merged_data.shape: 
        print("No data added!!!")

def combine_smp_rt_rc(date):
    data = pd.read_csv('../data/제주전력시장_시장전기가격_실시간가격.csv')
    
    smp_rt_rc = requests.get(f'https://research-api.solarkim.com/data/cmpt-2024/smp-rt-rc/{date.strftime('%Y-%m-%d')}', headers={
                            'Authorization': f'Bearer {API_KEY}'
                        }).json()
    
    smp_rt_rc = pd.DataFrame(smp_rt_rc)
    for column in smp_rt_rc.columns:
        data[column] = data[column].astype(smp_rt_rc[column].dtype)
    merged_data = pd.concat([data, smp_rt_rc], ignore_index=True)
    merged_data.to_csv('../data/제주전력시장_시장전기가격_실시간가격.csv', index=False)

    print(f'{date.strftime('%Y-%m-%d')} 실시간: ', data.shape, merged_data.shape)
    if data.shape == merged_data.shape: 
        print("No data added!!!")

def combine_elec_supply(date):
    data = pd.read_csv('../data/제주전력시장_현황데이터.csv')
    
    elec_supply = requests.get(f'https://research-api.solarkim.com/data/cmpt-2024/elec-supply/{date.strftime('%Y-%m-%d')}', headers={
                            'Authorization': f'Bearer {API_KEY}'
                        }).json()
    
    elec_supply = pd.DataFrame(elec_supply)
    for column in elec_supply.columns:
        data[column] = data[column].astype(elec_supply[column].dtype)
    merged_data = pd.concat([data, elec_supply], ignore_index=True)
    merged_data.to_csv('../data/제주전력시장_현황데이터.csv', index=False)

    print(f'{date.strftime('%Y-%m-%d')} 현황: ', data.shape, merged_data.shape)
    if data.shape == merged_data.shape: 
        print("No data added!!!")

In [ ]:
start_date = '2024-10-23'
end_date = '2024-11-13'

date_range = pd.date_range(start=start_date, end=end_date)

for date in date_range:
    combine_actual_weather(date)
    combine_weather_forecast(date)
    combine_smp_da(date)
    combine_smp_rt_rc(date)
    combine_elec_supply(date)